# Classification
Classify malware samples according to their labels using their reduced representations.

In [1]:
import tensorflow as tf
print(f"GPU is {'not ' if len(tf.config.list_physical_devices('GPU')) == 0 else ''}available")

2024-05-02 09:49:41.179249: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 09:49:41.715578: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU is available


2024-05-02 09:49:42.474976: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-02 09:49:42.504177: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-02 09:49:42.504219: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
import keras

# Data Processing

The `1-Data-Exploration.ipynb` file should have been run before this, and, as a result, the `processed-data.csv` file should have been generated.

In [3]:
import pandas as pd

In [4]:
raw_df = pd.read_csv("../data/processed-data.csv")
raw_df

,label,hash,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,TRICKBOT,f16631469eb35406ef4049d30c763cadda571b25bbdb45...,0.0,0.0,10.972300,2.885305,21.913673,17.404490,0.000000,3.189547,...,3.339754,11.345241,10.217216,0.0,8.646901,7.429351,0.0,0.589822,2.902267,7.847453
1,DARKKOMET,d31a7102cbc54447c251ba62760eb484fd0c9fbb8ea54f...,0.0,0.0,14.415304,4.064499,0.000000,0.000000,0.000000,4.895772,...,27.048859,0.000000,0.000000,0.0,5.451469,0.000000,0.0,0.000000,0.000000,0.000000
2,SALITY,e7fc7de574f44a966b198b7625bd6c595cad05bd669619...,0.0,0.0,3.820209,10.740729,19.335210,11.758485,4.437401,2.915592,...,0.000000,8.018223,8.416088,0.0,5.019428,11.775135,0.0,4.377707,6.175408,10.492670
3,VEBZENPAK,4519186b8fb2eaa847255087b44f918928c20e97c2fbea...,0.0,0.0,12.083241,5.210299,4.146192,0.000000,3.693242,0.000000,...,3.001556,5.981415,0.000000,0.0,11.143324,7.870592,0.0,1.047571,7.732687,9.557245
4,COBALTSTRIKE,dab956e9c864a84d12e8106a24ac3cf2950394152c62b6...,0.0,0.0,1.316455,4.444321,3.204388,0.000000,0.000000,0.000000,...,0.000000,1.693722,0.000000,0.0,0.000000,10.151480,0.0,3.554387,9.430399,8.058870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945,TRICKBOT,0c6aa0ae05d5fa8bf5a8ea95310be73ee60e55a0ce6864...,0.0,0.0,10.057159,10.175566,3.268311,0.000000,5.307435,6.613563,...,2.080789,0.000000,0.869716,0.0,12.583891,9.859617,0.0,0.117355,11.487420,5.953594
1946,TRICKBOT,020991f3452c6bdac2bd66e60174966bad7b7c201c7907...,0.0,0.0,13.221725,4.482760,2.375473,0.000000,4.333795,1.465900,...,2.147225,0.000000,3.746896,0.0,11.640550,2.526880,0.0,0.592801,4.686705,8.064886
1947,MANSABO,78514a632682d1c07ee4f782302bb6a74f2676f1a91b56...,0.0,0.0,15.995253,8.931343,0.000000,0.000000,0.000000,0.776199,...,19.978464,0.000000,0.000000,0.0,14.486612,7.519233,0.0,4.901716,0.000000,0.000000
1948,TRICKBOT,46401903e85a5c457490a6934ec4dc61fdf28df83af377...,0.0,0.0,19.242435,6.165306,0.000000,0.000000,0.000000,0.000000,...,12.655314,0.000000,4.641536,0.0,13.977305,9.326521,0.0,3.943131,0.305355,0.000000


For training the model, we don't need the `hash` of the sample.

In [5]:
df = raw_df.drop(columns=["hash"])
df

,label,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,dim-08,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,TRICKBOT,0.0,0.0,10.972300,2.885305,21.913673,17.404490,0.000000,3.189547,7.684439,...,3.339754,11.345241,10.217216,0.0,8.646901,7.429351,0.0,0.589822,2.902267,7.847453
1,DARKKOMET,0.0,0.0,14.415304,4.064499,0.000000,0.000000,0.000000,4.895772,0.000000,...,27.048859,0.000000,0.000000,0.0,5.451469,0.000000,0.0,0.000000,0.000000,0.000000
2,SALITY,0.0,0.0,3.820209,10.740729,19.335210,11.758485,4.437401,2.915592,5.255426,...,0.000000,8.018223,8.416088,0.0,5.019428,11.775135,0.0,4.377707,6.175408,10.492670
3,VEBZENPAK,0.0,0.0,12.083241,5.210299,4.146192,0.000000,3.693242,0.000000,0.000000,...,3.001556,5.981415,0.000000,0.0,11.143324,7.870592,0.0,1.047571,7.732687,9.557245
4,COBALTSTRIKE,0.0,0.0,1.316455,4.444321,3.204388,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.693722,0.000000,0.0,0.000000,10.151480,0.0,3.554387,9.430399,8.058870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945,TRICKBOT,0.0,0.0,10.057159,10.175566,3.268311,0.000000,5.307435,6.613563,0.000000,...,2.080789,0.000000,0.869716,0.0,12.583891,9.859617,0.0,0.117355,11.487420,5.953594
1946,TRICKBOT,0.0,0.0,13.221725,4.482760,2.375473,0.000000,4.333795,1.465900,0.000000,...,2.147225,0.000000,3.746896,0.0,11.640550,2.526880,0.0,0.592801,4.686705,8.064886
1947,MANSABO,0.0,0.0,15.995253,8.931343,0.000000,0.000000,0.000000,0.776199,0.000000,...,19.978464,0.000000,0.000000,0.0,14.486612,7.519233,0.0,4.901716,0.000000,0.000000
1948,TRICKBOT,0.0,0.0,19.242435,6.165306,0.000000,0.000000,0.000000,0.000000,0.000000,...,12.655314,0.000000,4.641536,0.0,13.977305,9.326521,0.0,3.943131,0.305355,0.000000


Separate into `X` and `y`.

In [6]:
X = df.drop(columns=["label"])
y = df["label"]

In [7]:
print(X.shape)
print(y.shape)

(1950, 32)
(1950,)


# Label Encoding and Train-Test Split

We need to encode the labels for the `y`.

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

We want to save the label encoder for use in the full model.

In [9]:
import pickle

with open("../models/classifier/label-encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

80% of the data will be saved for training, while 20% will be left for testing.

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1560, 32)
(390, 32)
(1560,)
(390,)


# Model Creation

Nothing too special here, we just use several dense layers for classification.

In [12]:
from keras import layers
from keras.models import Sequential

In [13]:
INITIAL_EPOCH = 0

def create_model():
    model = Sequential(name="Classifier")

    model.add(keras.Input((X.shape[1],), name="input"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(len(label_encoder.classes_), activation="softmax"))

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="adam",
        metrics=["sparse_categorical_accuracy"]
    )
    return model

In [14]:
model = create_model()
model.summary()

2024-05-02 09:49:42.800644: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-02 09:49:42.800727: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-02 09:49:42.800759: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-02 09:49:43.636405: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-02 09:49:43.636517: I external/local_xla/xla/stream_executor

Model: "Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 63)             │         4,095 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,295 (153.50 KB)

 Trainable params: 39,295 (153.50 KB)

 Non-trainable params: 0 (0.00 B)

Define callbacks.

In [15]:
import os

checkpoint_path = "../models/classifier/checkpoint.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [16]:
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=1, min_delta=1e-4)

Load latest checkpoint if there is one.

In [17]:
# INITIAL_EPOCH = 54
# model = keras.models.load_model(checkpoint_path)

Train the model.

In [18]:
NUM_EPOCHS = 100

model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    initial_epoch=INITIAL_EPOCH,
    epochs=NUM_EPOCHS,
    callbacks=[checkpointer, early_stopping]
)

Epoch 1/100


I0000 00:00:1714614584.444999   53153 service.cc:145] XLA service 0x7fc04c0046f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714614584.445050   53153 service.cc:153]   StreamExecutor device (0): Quadro P1000, Compute Capability 6.1
2024-05-02 09:49:44.464726: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-02 09:49:45.214386: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 1/39 ━━━━━━━━━━━━━━━━━━━━ 1:36 3s/step - loss: 5.0945 - sparse_categorical_accuracy: 0.0312

I0000 00:00:1714614586.401170   53153 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Epoch 1: val_loss improved from inf to 2.74740, saving model to ../models/classifier/checkpoint.keras
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 3.9837 - sparse_categorical_accuracy: 0.1367 - val_loss: 2.7474 - val_sparse_categorical_accuracy: 0.3878
Epoch 2/100
 1/39 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2.0273 - sparse_categorical_accuracy: 0.5000
Epoch 2: val_loss improved from 2.74740 to 2.27733, saving model to ../models/classifier/checkpoint.keras
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.6277 - sparse_categorical_accuracy: 0.3813 - val_loss: 2.2773 - val_sparse_categorical_accuracy: 0.4519
Epoch 3/100
 1/39 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.0705 - sparse_categorical_accuracy: 0.4375
Epoch 3: val_loss improved from 2.27733 to 1.92827, saving model to ../models/classifier/checkpoint.keras
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.1552 - sparse_categorical_accuracy: 0.4770 - val_loss: 1.9283 - val_sparse_categorical_accuracy: 0.5641
Epoch 4/100
 1/

Load the best performing model.

In [19]:
model = keras.models.load_model(checkpoint_path)

In [20]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f"Testing loss:      {test_loss:5.5f}")
print(f"Testing accuracy:  {test_acc:5.5f}")

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 1.4204 - sparse_categorical_accuracy: 0.6557
Testing loss:      1.45266
Testing accuracy:  0.64103


In [21]:
model.save("../models/classifier/classifier.keras")